In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

In [2]:
def load_and_preprocess_data(directory, target_size=(224, 224)):
    datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Normalize and split data

    train_generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,  # Resizing all images to this size
        batch_size=32,
        class_mode='binary',
        subset='training'
    )
    
    validation_generator = datagen.flow_from_directory(
        directory,
        target_size=target_size,  # Resizing all images to this size
        batch_size=32,
        class_mode='binary',
        subset='validation'
    )
    return train_generator, validation_generator

In [4]:
def build_model():
    base_model = ResNet50(weights='imagenet', include_top=False)
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    predictions = Dense(1, activation='sigmoid')(x)
    model = Model(inputs=base_model.input, outputs=predictions)
    
    # Freeze all layers of the base model
    for layer in base_model.layers:
        layer.trainable = False
    
    # Update this line with the corrected parameter name
    model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [5]:
model = build_model()

In [8]:
train_data, val_data = load_and_preprocess_data('../DATA')

Found 1450 images belonging to 2 classes.
Found 362 images belonging to 2 classes.


In [20]:
model.fit(train_data, epochs=16, validation_data=val_data)

Epoch 1/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 28s 600ms/step - accuracy: 0.8023 - loss: 0.4122 - val_accuracy: 0.6077 - val_loss: 0.7671
Epoch 2/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 592ms/step - accuracy: 0.8119 - loss: 0.4222 - val_accuracy: 0.6133 - val_loss: 0.7714
Epoch 3/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 590ms/step - accuracy: 0.8120 - loss: 0.4033 - val_accuracy: 0.5718 - val_loss: 0.7757
Epoch 4/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 590ms/step - accuracy: 0.8222 - loss: 0.3956 - val_accuracy: 0.6077 - val_loss: 0.7668
Epoch 5/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 591ms/step - accuracy: 0.8248 - loss: 0.3840 - val_accuracy: 0.6133 - val_loss: 0.7677
Epoch 6/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 591ms/step - accuracy: 0.8195 - loss: 0.3893 - val_accuracy: 0.6022 - val_loss: 0.7688
Epoch 7/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 590ms/step - accuracy: 0.8276 - loss: 0.3975 - val_accuracy: 0.5801 - val_loss: 0.7714
Epoch 8/16
46/46 ━━━━━━━━━━━━━━━━━━━━ 27s 592ms/step - accuracy: 0.8063 - loss: 0.4190 - val_accu

In [59]:
model.save_weights('../MODELs/classification_resnet50.weights.h5')

In [14]:
model.load_weights('../MODELs/classification_resnet50.weights.h5')

In [15]:
def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = image.load_img(img_path, target_size=target_size)
    img_array = image.img_to_array(img)
    img_array_expanded = np.expand_dims(img_array, axis=0)  # Model expects a batch
    return preprocess_input(img_array_expanded)
def predict_image(model, processed_image):
    prediction = model.predict(processed_image)
    return prediction
def interpret_prediction(prediction, threshold=0.5):
    if prediction[0][0] > threshold:
        return "Positive (wearing uniform)"
    else:
        return "Negative (not wearing uniform)"

In [21]:
img_path = '../DATA/positive/person_patch_20240427_203351_137164_0.77.png'
processed_image = load_and_preprocess_image(img_path)
prediction = predict_image(model, processed_image)
result = interpret_prediction(prediction)
print(result)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step
Negative (not wearing uniform)
